In [5]:
import sqlite3
con = sqlite3.connect('test.db')

In [28]:
import sqlite3
con = sqlite3.connect('test.db')

cur = con.cursor()
# Create table
#cur.execute('''CREATE TABLE TaskInstance
#               (dag_id text, task_id text, execution_date text, instance real, state text)''')

#cur.execute('''CREATE TABLE TaskReschedule
#               (dag_id text, task_id text, execution_date text, reschedule_date text)''')

#cur.execute('''CREATE TABLE DagRun
#               (id, dag_id, execution_date, state, run_id, external_trigger, conf, end_date, start_date, run_type, last_scheduling_decision, dag_hash, creating_job_id)''')


cur.execute("INSERT INTO DagRun VALUES ('1', 'smoke_test_env_vars', '2021-04-1408:40:00.000000', 'failed', 'scheduled__2021-04-14T08:40:00+00:00', '0', '?}?.', '2021-04-1408:56:47.747434', '2021-04-1408:49:57.150681', 'scheduled', '2021-04-1408:56:47.742612', '482a7b9b8d5bb41b69b6b2e707531839', '1')")
cur.execute("INSERT INTO DagRun VALUES ('2', 'smoke_test_env_vars', '2021-04-1409:40:00.000000', 'success', 'scheduled__2021-04-14T08:40:00+00:00', '0', '?}?.', '2021-04-1408:56:47.747434', '2021-04-1408:49:57.150681', 'scheduled', '2021-04-1408:56:47.742612', '482a7b9b8d5bb41b69b6b2e707531839', '1')")
cur.execute("INSERT INTO DagRun VALUES ('3', 'smoke_test_env_vars2', '2021-04-1409:40:00.000000', 'success', 'scheduled__2021-04-14T08:40:00+00:00', '0', '?}?.', '2021-04-1408:56:47.747434', '2021-04-1408:49:57.150681', 'scheduled', '2021-04-1408:56:47.742612', '482a7b9b8d5bb41b69b6b2e707531839', '1')")
cur.execute("INSERT INTO DagRun VALUES ('4', 'smoke_test_env_vars', '2021-04-1409:40:00.000000', 'success', 'scheduled__2021-04-14T08:40:00+00:00', '0', '?}?.', '2021-04-1409:56:47.747434', '2021-04-1408:49:57.150681', 'scheduled', '2021-04-1408:56:47.742612', '482a7b9b8d5bb41b69b6b2e707531839', '1')")
# Insert a row of data
cur.execute("INSERT INTO TaskInstance VALUES ('a','a','2021-03-31', 1, 'UP_FOR_RESCHEDULE')")
cur.execute("INSERT INTO TaskInstance VALUES ('b','b','2021-03-31', 1, 'UP_FOR_RESCHEDULE')")

cur.execute("INSERT INTO TaskReschedule VALUES ('a','a','2021-03-31', '2021-03-31 00:00:10')")
cur.execute("INSERT INTO TaskReschedule VALUES ('a','a','2021-03-31', '2021-03-31 00:00:20')")
cur.execute("INSERT INTO TaskReschedule VALUES ('a','a','2021-03-31', '2021-03-31 00:00:30')")
cur.execute("INSERT INTO TaskReschedule VALUES ('b','b','2021-03-31', '2021-03-31 00:00:30')")

# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

In [8]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///test.db', echo=True)

In [25]:
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.sql import and_, func
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

class TaskInstance(Base):
    __tablename__ = 'TaskInstance'

    dag_id = Column(String, primary_key=True)
    task_id = Column(String, primary_key=True)
    execution_date = Column(String, primary_key=True)
    instance = Column(Float)
    state = Column(String)
    
class TaskReschedule(Base):
    __tablename__ = 'TaskReschedule'

    dag_id = Column(String, primary_key=True)
    task_id = Column(String, primary_key=True)
    execution_date = Column(String, primary_key=True)
    reschedule_date = Column(String, primary_key=True)

class DagRun(Base):
    __tablename__ = 'DagRun'

    id = Column(String)
    dag_id = Column(String, primary_key=True)
    task_id = Column(String, primary_key=True)
    execution_date = Column(String, primary_key=True)
    reschedule_date = Column(String, primary_key=True)
    state = Column(String)
    run_id = Column(String)
    external_trigger = Column(String)
    conf = Column(String)
    end_date = Column(String)
    start_date = Column(String)
    run_type = Column(String)
    last_scheduling_decision = Column(String)
    dag_hash = Column(String)
    creating_job_id = Column(String)
    
Base.metadata.create_all(engine)

2021-04-14 13:35:22,787 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("TaskInstance")
2021-04-14 13:35:22,789 INFO sqlalchemy.engine.base.Engine ()
2021-04-14 13:35:22,793 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("TaskReschedule")
2021-04-14 13:35:22,796 INFO sqlalchemy.engine.base.Engine ()
2021-04-14 13:35:22,799 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("DagRun")
2021-04-14 13:35:22,801 INFO sqlalchemy.engine.base.Engine ()


In [10]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()
last_ts_to_clean = "2021-03-31 00:00:20"

In [15]:
query = session.query(
    TaskInstance,
    func.max(TaskReschedule.reschedule_date)
).join(
    TaskReschedule, and_(
        TaskReschedule.dag_id == TaskInstance.dag_id,
        TaskReschedule.task_id == TaskInstance.task_id,
        TaskReschedule.execution_date == TaskInstance.execution_date
    )
).filter(
    TaskInstance.state == "UP_FOR_RESCHEDULE",
).group_by(
  TaskInstance
).having(
    func.max(TaskReschedule.reschedule_date) < last_ts_to_clean
).all()

2021-04-14 11:54:13,152 INFO sqlalchemy.engine.base.Engine SELECT "TaskInstance".dag_id AS "TaskInstance_dag_id", "TaskInstance".task_id AS "TaskInstance_task_id", "TaskInstance".execution_date AS "TaskInstance_execution_date", "TaskInstance".instance AS "TaskInstance_instance", "TaskInstance".state AS "TaskInstance_state", max("TaskReschedule".reschedule_date) AS max_1 
FROM "TaskInstance" JOIN "TaskReschedule" ON "TaskReschedule".dag_id = "TaskInstance".dag_id AND "TaskReschedule".task_id = "TaskInstance".task_id AND "TaskReschedule".execution_date = "TaskInstance".execution_date 
WHERE "TaskInstance".state = ? GROUP BY "TaskInstance".dag_id, "TaskInstance".task_id, "TaskInstance".execution_date, "TaskInstance".instance, "TaskInstance".state 
HAVING max("TaskReschedule".reschedule_date) < ?
2021-04-14 11:54:13,153 INFO sqlalchemy.engine.base.Engine ('UP_FOR_RESCHEDULE', '2021-03-31 00:00:20')


In [12]:
print(query)

SELECT "TaskInstance".dag_id AS "TaskInstance_dag_id", "TaskInstance".task_id AS "TaskInstance_task_id", "TaskInstance".execution_date AS "TaskInstance_execution_date", "TaskInstance".instance AS "TaskInstance_instance", "TaskInstance".state AS "TaskInstance_state", max("TaskReschedule".reschedule_date) AS max_1 
FROM "TaskInstance" JOIN "TaskReschedule" ON "TaskReschedule".dag_id = "TaskInstance".dag_id AND "TaskReschedule".task_id = "TaskInstance".task_id AND "TaskReschedule".execution_date = "TaskInstance".execution_date 
WHERE "TaskInstance".state = ? GROUP BY "TaskInstance".dag_id, "TaskInstance".task_id, "TaskInstance".execution_date, "TaskInstance".instance, "TaskInstance".state 
HAVING max("TaskReschedule".reschedule_date) < ?


In [13]:
query.all()

2021-04-14 11:53:29,782 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-04-14 11:53:29,786 INFO sqlalchemy.engine.base.Engine SELECT "TaskInstance".dag_id AS "TaskInstance_dag_id", "TaskInstance".task_id AS "TaskInstance_task_id", "TaskInstance".execution_date AS "TaskInstance_execution_date", "TaskInstance".instance AS "TaskInstance_instance", "TaskInstance".state AS "TaskInstance_state", max("TaskReschedule".reschedule_date) AS max_1 
FROM "TaskInstance" JOIN "TaskReschedule" ON "TaskReschedule".dag_id = "TaskInstance".dag_id AND "TaskReschedule".task_id = "TaskInstance".task_id AND "TaskReschedule".execution_date = "TaskInstance".execution_date 
WHERE "TaskInstance".state = ? GROUP BY "TaskInstance".dag_id, "TaskInstance".task_id, "TaskInstance".execution_date, "TaskInstance".instance, "TaskInstance".state 
HAVING max("TaskReschedule".reschedule_date) < ?
2021-04-14 11:53:29,790 INFO sqlalchemy.engine.base.Engine ('UP_FOR_RESCHEDULE', '2021-03-31 00:00:20')


[]

In [14]:
session.query(
    TaskInstance,
    func.max(TaskReschedule.reschedule_date)
).join(
    TaskReschedule, and_(
        TaskReschedule.dag_id == TaskInstance.dag_id,
        TaskReschedule.task_id == TaskInstance.task_id,
        TaskReschedule.execution_date == TaskInstance.execution_date
    )
).filter(
    TaskInstance.state == "UP_FOR_RESCHEDULE",
).group_by(
  TaskInstance
).all()

2021-04-14 11:53:37,417 INFO sqlalchemy.engine.base.Engine SELECT "TaskInstance".dag_id AS "TaskInstance_dag_id", "TaskInstance".task_id AS "TaskInstance_task_id", "TaskInstance".execution_date AS "TaskInstance_execution_date", "TaskInstance".instance AS "TaskInstance_instance", "TaskInstance".state AS "TaskInstance_state", max("TaskReschedule".reschedule_date) AS max_1 
FROM "TaskInstance" JOIN "TaskReschedule" ON "TaskReschedule".dag_id = "TaskInstance".dag_id AND "TaskReschedule".task_id = "TaskInstance".task_id AND "TaskReschedule".execution_date = "TaskInstance".execution_date 
WHERE "TaskInstance".state = ? GROUP BY "TaskInstance".dag_id, "TaskInstance".task_id, "TaskInstance".execution_date, "TaskInstance".instance, "TaskInstance".state
2021-04-14 11:53:37,420 INFO sqlalchemy.engine.base.Engine ('UP_FOR_RESCHEDULE',)


[(<__main__.TaskInstance at 0x7fb58c2042b0>, '2021-03-31 00:00:30'),
 (<__main__.TaskInstance at 0x7fb58c204310>, '2021-03-31 00:00:30')]

In [37]:
session.query(
        DagRun.dag_id,
        func.max(DagRun.end_date).label("end_date")
    ).filter(
        DagRun.state == "success"
    ).group_by(
        DagRun.dag_id
    ).all()

2021-04-14 13:43:32,298 INFO sqlalchemy.engine.base.Engine SELECT "DagRun".dag_id AS "DagRun_dag_id", max("DagRun".end_date) AS end_date 
FROM "DagRun" 
WHERE "DagRun".state = ? GROUP BY "DagRun".dag_id
2021-04-14 13:43:32,301 INFO sqlalchemy.engine.base.Engine ('success',)


[('smoke_test_env_vars', '2021-04-1409:56:47.747434'),
 ('smoke_test_env_vars2', '2021-04-1408:56:47.747434')]